<a href="https://colab.research.google.com/github/badcortex/opt4ds/blob/master/folding_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order to run the following notebook it is necessary to install the *Pyomo* library.  


In [0]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# The RNA Folding problem 

**The Simple RNA Folding problem** : Given the nucleotide sequence *s* of a RNA molecule, find a *nested pairing* that pairs the *maximum* number of nucleotides, compared to any other nested pairing. 

### A toy example
The ILP formulation for the Simple RNA Folding problem will have one binary variable, called $P(i,j)$, for each pair $(i,j)$ of positions in *s*, where $i < j$. The value $P(i,j)$ given by a feasible solution to the ILP formulation whether or not the nucleotide in $i$ of *s* will be paired with the nucleotide in position $j$ of *s*. 

1. **The objective funtion** : $$ Maximize \sum_{i<j} P(i,j) $$ 
2. **The inequalities** : For every pair $(i,j)$ of positions in *s* that do not have complementary characters $$ P(i,j) = 0 $$ 
For each position $j$ in *s* $$ \sum_{k < j} P(k,j) + \sum_{k > j} P(j,k) \leq 1 $$ For every choice of four positions $i < i'< j < j'$
$$ P(i,j)+P(i',j') \leq 1 $$

In [0]:
from pyomo.environ import * 

rna = ["A","C","U","G","U"]
rna_length = len(rna)

# Creazione del modello
model = ConcreteModel() 

# Indici 
model.I = RangeSet(1, rna_length)
model.J = RangeSet(1, rna_length + 1)

# Variabili decisionali 
model.P = Var(model.I, model.J, within=Binary)

# Funzione obiettivo 
model.Obj = Objective(expr=sum(model.P[i,j] for i in model.I for j in range(i+1,rna_length+1)), sense=maximize)
 
# Vincoli 
model.ComPairs = ConstraintList()
for i in model.I:
  for j in range(i+1,rna_length+1):
    if (rna[i-1],rna[j-1]) in [("A","U"),("U","A"),("C","G"),("G","C")]:
      model.ComPairs.add(model.P[i,j] == 1)
    else:
      model.ComPairs.add(model.P[i,j] == 0)

model.PairPos = ConstraintList()
for i in model.I:
  expr = 0
  for j in range(1, i):
    expr += model.P[j,i]
  for j in range(i+1, rna_length+1):
    expr += model.P[i,j]
  model.PairPos.add(expr <= 1) 

model.NoCross = ConstraintList()
for h in range(1, rna_length-2):
  for i in range(h+1, rna_length-1):
    for j in range(i+1, rna_length):
      for k in range(j+1, rna_length+1):
         model.NoCross.add(model.P[h,j] + model.P[i,k] <= 1) 


In [46]:
# Debug
model.Obj.pprint()
model.ComPairs.pprint()
model.PairPos.pprint()
model.Obj.display()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : P[1,2] + P[1,3] + P[1,4] + P[1,5] + P[2,3] + P[2,4] + P[2,5] + P[3,4] + P[3,5] + P[4,5]
ComPairs : Size=10, Index=ComPairs_index, Active=True
    Key : Lower : Body   : Upper : Active
      1 :   0.0 : P[1,2] :   0.0 :   True
      2 :   1.0 : P[1,3] :   1.0 :   True
      3 :   0.0 : P[1,4] :   0.0 :   True
      4 :   1.0 : P[1,5] :   1.0 :   True
      5 :   0.0 : P[2,3] :   0.0 :   True
      6 :   1.0 : P[2,4] :   1.0 :   True
      7 :   0.0 : P[2,5] :   0.0 :   True
      8 :   0.0 : P[3,4] :   0.0 :   True
      9 :   0.0 : P[3,5] :   0.0 :   True
     10 :   0.0 : P[4,5] :   0.0 :   True
PairPos : Size=5, Index=PairPos_index, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :  -Inf : P[1,2] + P[1,3] + P[1,4] + P[1,5] :   1.0 :   True
      2 :  -Inf : P[1,2] + P[2,3] + P[2,4] + P[2,5] :   1.0 :   True
      3 :  -Inf : P[1,3] + 

In [0]:
# Solve the model
sol = SolverFactory('glpk').solve(model)

# Basic info about the solution process
for info in sol['Solver']:
    print(info)

# Report solution value
print("Optimal solution value: z =", model.Obj())
print("Decision variables:")
for i in range(1,rna_length):
    for j in range(i,rna_length+1):
        print("P({},{}) = {}".format(i, j, model.P[i,j]()))